Preprocess

In [13]:
import io
import re

# 발라드 크롤링 데이터 전처리
path_ballad = './song_folder/lyric_data_ballad.txt'
with io.open(path_ballad, encoding='utf-8') as f_ballad: #텍스트 파일을 f에 읽어옴 (한국어 인코딩인 CP949사용 )
    text_ballad = f_ballad.read().lower() #f에 내용들을 소문자화

text_ballad = re.sub(r'\n', ' ', text_ballad) #줄바꿈을 공백으로
text_ballad = re.sub(r'-', ' ', text_ballad) # '-'를 공백으로
text_ballad = re.sub('[^a-zA-Z0-9ㄱ-ㅣ가-힣]',' ',text_ballad) # 한국어와 영어, 숫자 제외 다른 언어 제거
text_ballad = ' '.join(text_ballad.split()) #여러개의 공백을 제거하고 하나의 문자열로 만듦

# 힙합 크롤링 데이터 전처리
path_hiphop = './song_folder/lyric_data_hiphop.txt'
with io.open(path_hiphop, encoding='utf-8') as f_hiphop: #텍스트 파일을 f에 읽어옴 (한국어 인코딩인 CP949사용 )
    text_hiphop = f_hiphop.read().lower() #f에 내용들을 소문자화

text_hiphop = re.sub(r'\n', ' ', text_hiphop) #줄바꿈을 공백으로
text_hiphop = re.sub(r'-', ' ', text_hiphop) # '-'를 공백으로
text_hiphop = re.sub('[^a-zA-Z0-9ㄱ-ㅣ가-힣]',' ',text_hiphop) # 한국어와 영어, 숫자 제외 다른 언어 제거
text_hiphop = ' '.join(text_hiphop.split()) #여러개의 공백을 제거하고 하나의 문자열로 만듦

print('ballad corpus length:', len(text_ballad)) 
print('hiphop corpus length:', len(text_hiphop)) 

# 전처리 후 텍스트파일로 추출
f1 = open('./song_folder/lyric_data_ballad_preprocessed.txt', 'a', encoding='utf-8')
f1.write(text_ballad) 
f1.close()
f2 = open('./song_folder/lyric_data_hiphop_preprocessed.txt', 'a', encoding='utf-8')
f2.write(text_hiphop) 
f2.close()

ballad corpus length: 717967
hiphop corpus length: 1525471


Vectorize Sentences

In [14]:
# KOGPT2의 transformers를 불러온다.
# transformers는 자연어 처리 모델로, RNN을 사용하지 않고 Attention 만으로도 충분이 seq2seq 자연어 처리를 할 수 있다.

from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer = tokenizer,
        mlm = mlm,
    )
    return data_collator

def train(train_file_path,model_name,
         output_dir,
         overwrite_output_dir,
         per_device_train_batch_size,
         num_train_epochs,
         save_steps):
    
    # Text를 여러개의 Token으로 분류하며 보통 공백, 구두점, 특수문자 등으로 분류한다.
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name,
                bos_token='<s>', eos_token='</s>', unk_token='<unk>',
                pad_token='<pad>', mask_token='<mask>')
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)
    
    tokenizer.save_pretrained(output_dir, legacy_format=False)
    
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    model.save_pretrained(output_dir)
    
    training_args = TrainingArguments(
            output_dir=output_dir,
            overwrite_output_dir=overwrite_output_dir,
            per_device_train_batch_size=per_device_train_batch_size,
            num_train_epochs=num_train_epochs,
    )
    
    trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=train_dataset,
    )
    
    trainer.train()
    trainer.save_model()
    
model_name = 'skt/kogpt2-base-v2'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 5.0
save_steps = 500

# 발라드 모델 학습
train(
    train_file_path='./song_folder/lyric_data_ballad_preprocessed.txt',
    model_name=model_name,
    output_dir='./models_ballad',
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

#힙합 모델 학습
train(
    train_file_path='./song_folder/lyric_data_hiphop_preprocessed.txt',
    model_name=model_name,
    output_dir='./models_hiphop',
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
***** Running training *****
  Num examples = 2598
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1625


Saving model checkpoint to ./models_ballad\checkpoint-500
Configuration saved in ./models_ballad\checkpoint-500\config.json
Model weights saved in ./models_ballad\checkpoint-500\pytorch_model.bin
Saving model checkpoint to ./models_ballad\checkpoint-1000
Configuration saved in ./models_ballad\checkpoint-1000\config.json
Model weights saved in ./models_ballad\checkpoint-1000\pytorch_model.bin
Saving model checkpoint to ./models_ballad\checkpoint-1500
Configuration saved in ./models_ballad\checkpoint-1500\config.json
Model weights saved in ./models_ballad\checkpoint-1500\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./models_ballad
Configuration saved in ./models_ballad\config.json
Model weights saved in ./models_ballad\pytorch_model.bin
loading file https://huggingface.co/skt/kogpt2-base-v2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/skt/kogpt2-base-v2/resolve/m

Saving model checkpoint to ./models_hiphop\checkpoint-500
Configuration saved in ./models_hiphop\checkpoint-500\config.json
Model weights saved in ./models_hiphop\checkpoint-500\pytorch_model.bin
Saving model checkpoint to ./models_hiphop\checkpoint-1000
Configuration saved in ./models_hiphop\checkpoint-1000\config.json
Model weights saved in ./models_hiphop\checkpoint-1000\pytorch_model.bin
Saving model checkpoint to ./models_hiphop\checkpoint-1500
Configuration saved in ./models_hiphop\checkpoint-1500\config.json
Model weights saved in ./models_hiphop\checkpoint-1500\pytorch_model.bin
Saving model checkpoint to ./models_hiphop\checkpoint-2000
Configuration saved in ./models_hiphop\checkpoint-2000\config.json
Model weights saved in ./models_hiphop\checkpoint-2000\pytorch_model.bin
Saving model checkpoint to ./models_hiphop\checkpoint-2500
Configuration saved in ./models_hiphop\checkpoint-2500\config.json
Model weights saved in ./models_hiphop\checkpoint-2500\pytorch_model.bin
Saving m

In [86]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import warnings

warnings.filterwarnings(action='ignore') 


def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

def load_tokenizer(tokenizer_path):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
    return tokenizer

#발라드 텍스트 생성
def generate_text_ballad(sequence, max_length):
    model_path = './models_ballad'
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence},', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        repetition_penalty=1.1, #반복 제거
        #no_repeat_ngram_size=4,
        #temperature=100.0,
        max_length=max_length,
        pad_token_id=model.config.pad_token_id,
        top_k=50, #토큰 확률분포에서 확률값이 가장 높은 k개 중에서 선택
        top_p=0.95 #확률값이 높은 순서대로 내림차순 정렬을 한 뒤 누적 확률값이  p  이하인 단어들 가운데 선택
    )
    return tokenizer.decode(final_outputs[0], skip_special_tokens=True)

def generate_text_hiphop(sequence, max_length):
    model_path = './models_hiphop'
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence},', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        repetition_penalty=1.1, #반복 제거
        max_length=max_length,
        pad_token_id=model.config.pad_token_id,
        top_k=50, #토큰 확률분포에서 확률값이 가장 높은 k개 중에서 선택
        top_p=0.95 #확률값이 높은 순서대로 내림차순 정렬을 한 뒤 누적 확률값이  p  이하인 단어들 가운데 선택
    )
    return tokenizer.decode(final_outputs[0], skip_special_tokens=True)
    
while(True):
    your_name = input("당신은 누구인가요? ")
    sequence = input("\n안녕하세요. 오늘 당신의 하루를 노래로 만들어 드릴게요.\n\n"+your_name + "님의 이야기를 들려주세요 : ")

    max_len = len(sequence)*3 + 100-len(sequence)*2
    print("======================================================\n노래를 시작합니다.\n======================================================")
    break;
      
result = [[0 for col in range(2)] for row in range(5)]
    
for i in range(5):
    result[i][0] = generate_text_ballad(sequence,max_len)
    result[i][1] = generate_text_hiphop(sequence,max_len)
    
print("\n노래 가사 생성이 완료되었습니다!")

당신은 누구인가요? 산타클로스

안녕하세요. 오늘 당신의 하루를 노래로 만들어 드릴게요.

산타클로스님의 이야기를 들려주세요 : 메리크리스마스


loading configuration file ./models_ballad\config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cl

노래를 시작합니다.


All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at ./models_ballad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file ./models_ballad\added_tokens.json. We won't load it.
loading file None
loading file ./models_ballad\special_tokens_map.json
loading file ./models_ballad\tokenizer_config.json
loading file ./models_ballad\tokenizer.json
loading configuration file ./models_hiphop\config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2

loading weights file ./models_hiphop\pytorch_model.bin
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at ./models_hiphop.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file ./models_hiphop\added_tokens.json. We won't load it.
loading file None
loading file ./models_hiphop\special_tokens_map.json
loading file ./models_hiphop\tokenizer_config.json
loading file ./models_hiphop\tokenizer.json
loading configuration file ./models_ballad\config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_


노래 가사 생성이 완료되었습니다!


In [87]:
import pandas as pd

df = pd.DataFrame(result)
df.columns = ['발라드', '힙합']
pd.set_option('display.colheader_justify','right')
pd.set_option('display.max_columns', None) # 모든 열 출력
pd.set_option('display.max_colwidth',None) # 열의 표시되는 글자수 제한 해제

In [88]:
print(your_name+"님의 오늘의 이야기를 노래로 만들었어요 ♬")
dfStyler = df.style.set_properties(**{'text-align': 'left'}) # 텍스트 왼쪽 정렬
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]) # label 제목 가운데 정렬

산타클로스님의 오늘의 이야기를 노래로 만들었어요 ♬
